# Convolutional Neural Networks: Step by Step

Welcome to Course 4's first assignment! In this notebook, you will implement the functions required to build a convolutional neural network. 

**After this assignment you will be able to:**
- Understand how convolution and pooling layers work
- Build the building blocks of a convolutional neural network
- Implement the forward and backward propagations for convolutional and pooling layers

**Notations**:
- Superscript $[l]$ denotes an object of the $l^{th}$ layer. 
    - Example: $a^{[L]}$ is the $L^{th}$ layer activation. $W^{[L]}$ and $b^{[L]}$ are the $L^{[th]}$ layer parameters.
- Superscript $(i)$ denotes an object from the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example.
- Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $z^{[l]}_i$ denotes the $i^{th}$ entry of the output vector of the ($l^{th}$ layer's linear unit).

Let's get started!

## 1 - Packages

Let's first import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [matplotlib](http://matplotlib.org) is a library to plot graphs in Python.
- np.random.seed(1) is used to keep all the random function calls consistent. It will help us grade your work.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Outline of the Assignment

You will be implementing the building blocks of a convolutional neural network! Each function you will implement will have detailed instructions that will walk you through all the steps needed to complete it:

- Convolution functions, includes:
    - Zero Padding
    - Convolve window 
    - Convolution forward
    - Convolution backward (optional)
- Pooling functions, includes:
    - Pooling forward
    - Create mask
    - Distribute value
    - Pooling backward (optional)
    
These functions are the ones you will use in tensorflow in the next assignment to build the following model:

<img src="images/model.png" style="width:800px;height:300px;">

We will discuss this model in the next assignements. For now, you are going to build functions in numpy to understand the mechanics behind the implementation.

**Note** that for every forward function, there is its corresponding backward equivalent. Hence, at every step of your forward module you will be storing some parameters in a cache. These parameters are used to compute gradients during backpropagation. 

## 3 - Convolutional Neural Networks

Although programming frameworks make convolutions easy to use, they remain one of the hardest concepts to understand in Deep Learning. A convolution layer transforms an input volume into an output volume of different size (usually smaller but deeper) as shown below. 

<img src="images/conv_nn.png" style="width:350px;height:200px;">

In this part, you are going to build every step of the convolution layer. You will first implement two helper functions; one for zero padding and the other for computing the convolution on a window of values. 

### 3.1 - Zero-Padding

Zero-padding is a technique used in convolutional layers and refers to adding zeros around the border of an image (as shown in the following figure). [change this to 4 by 4]

<img src="images/PAD.png" style="width:600px;height:400px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **Zero-Padding**<br> Image (3 channels, RGB) being padded with a vertical padding of 2 and a horizontal padding of 2 </center></caption>

The main benefits of padding are the following:

- It slows down the volume reduction between layers and thus allows us to build deeper networks.

- It improves performance because it helps us keep the information at the border of an image. Without padding, we might not be able to preserve all the pixels at the edge of an image.

- It can preserve the dimensions.


**Exercise**: Implement the following function which pads all the images of a batch X with zeros. [Use np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html).

In [ ]:
# GRADED FUNCTION: zero_pad

def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X
    
    Argument:
    X -- python numpy array of shape (m, H, W, C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal axes
    
    Returns:
    X_pad -- padded image of shape (H + 2*pad, W + 2*pad, C)
    """
    
    ### START CODE HERE ### (≈ 1 line)
    X_pad = None
    ### END CODE HERE ###
    
    return X_pad

In [ ]:
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print ("x.shape =", x.shape)
print ("x_pad.shape =", x_pad.shape)
print ("x[1,1] =", x[1,1])
print ("x_pad[1,1] =", x_pad[1,1])

**Expected Output**:

<table>
    <tr>
        <td>
            **x.shape**:
        </td>
        <td>
           (4, 3, 3, 2)
        </td>
    </tr>
        <tr>
        <td>
            **x_pad.shape**:
        </td>
        <td>
           (4, 7, 7, 2)
        </td>
    </tr>
        <tr>
        <td>
            **x[1,1]**:
        </td>
        <td>
           [[-1.21784605  0.05769209]
 [ 0.60835538 -0.35169188]
 [ 0.36866569  0.10463232]]
        </td>
    </tr>
        <tr>
        <td>
            **x_pad[1,1]**:
        </td>
        <td>
           [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
        </td>
    </tr>

</table>

### 3.2 - Convolve a window

In this part, you will learn what "convolve a window" means and implement it. First let's define the term window. A convolution unit:

- Takes an input volume 
- Applies the convolution operation
- Outputs another volume (usually of different size)


When you convolve a 2d image around a window, you are applying a filter to an input by 
convolution. In the case of a 2d image, the convolution operation can be seen as a sliding window (containing the parameters) over the input volume.

<img src="images/Convolution_schematic.gif" style="width:500px;height:300px;">
<caption><center> <u> <font color='purple'> **Figure 2** </u><font color='purple'>  : **Convolution operation**<br> </center></caption>

Imagine that the matrix on the left represents a black and white image. Each entry corresponds to one pixel, 0 for black and 1 for white (typically it’s between 0 and 255 for grayscale images). The sliding window is called a kernel, filter, or feature detector. Here we use a 3×3 filter, multiply its values element-wise with the original matrix, then sum them up. To get the full convolution we do this for each element by sliding the filter over the whole matrix.

**Exercise**: Implement convolve_window(). [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.sum.html)

In [ ]:
# GRADED FUNCTION: convolve_window

def convolve_window(a_slice, W, b):
    """
    Convolves a window of parameters W on a single slice (a_slice) of the output activation of the previous layer.
    
    Arguments:
    a_slice -- slice of input data of shape (Hp, Wp, Cp)
    W -- Weight parameters contained in a window-matrix of shape (Hp, Wp, Cp)
    b -- Bias parameters contained in a window-matrix of shape (1, 1, Cp)
    
    Returns:
    Z -- a scalar value, result of convolving the sliding window (W, b) on a slice x of the input data
    """

    ### START CODE HERE ### (≈ 2 lines of code)
    # Element-wise product between a_slice and W. Add bias.
    s = None
    # Sum over all entries of the volume
    Z = None
    ### END CODE HERE ###

    return Z

In [ ]:
np.random.seed(1)
a_slice = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 3)

Z = convolve_window(a_slice, W, b)
print("Z =", Z)

**Expected Output**:
<table>
    <tr>
        <td>
            **Z**
        </td>
        <td>
            -21.3793583781
        </td>
    </tr>

</table>

### 3.3 - Convolutional Neural Networks - Forward pass

In the forward pass, you will take many filters and convolve them around the input. Each 'convolution' gives you a 2D matrix output. You will then stack these outputs to get a cube: 

<center>
<video width="620" height="440" src="images/conv_kiank.mp4" type="video/mp4" controls>
</video>
</center>

**Exercise**: Implement the function below to convolve the filters W around an input activation A. This function takes in A, activations output of the previous layer (for a batch of m inputs), F filters or weights denoted by W, and a bias vector denoted by b, where each filter has its own (single) bias. Finally you also have access to the hyperparameters dictionary which contain the stride and the padding.

In [ ]:
# GRADED FUNCTION: conv_forward

def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function
    
    Arguments:
    A_prev -- output activations of the previous layer, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f,f,n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"
        
    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = None
    
    # Retrieve information from "hparameters"
    stride = None
    pad = None
    
    # Compute the dimensions of the CONV output volume using the formula seen in lecture. Hint: use int() to floor.
    n_H = None
    n_W = None
    
    # Initialize the output volume Z with zeros.
    Z = None
    A_prev_pad = None
    
    for i in range(m):                                  # loop over the batch of training examples
        a_prev = A_prev_pad[i,:,:,:]                    # Select ith training example's activation
        for h in range(n_H):                            # loop over vertical axis of the output volume
            for w in range(n_W):                        # loop over horizontal axis of the output volume
                for c in range(n_C):                    # loop over channels (= #filters) of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Use the corners to define the slice
                    a_slice = None
                    
                    # Convolve the window to get back one output neuron
                    Z[i, h, w, c] = None
                                        
    ### END CODE HERE ###
    
    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

In [ ]:
np.random.seed(1)
X = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,3,8)
hparameters = {"pad" : 2,
               "stride": 1}

Z, cache = conv_forward(X, W, b, hparameters)
print("Z's mean =", np.mean(Z))
print("cache[0][1][2][3] =", cache[0][1][2][3])

**Expected Output**:

<table>
    <tr>
        <td>
            **Z's mean**
        </td>
        <td>
            2.5961295689
        </td>
    </tr>
    <tr>
        <td>
            **cache[0][1][2][3]**
        </td>
        <td>
            [-0.20075807  0.18656139  0.41005165]
        </td>
    </tr>

</table>


In practice, a CONV layer would also contain an activation in which case we would add the following line of code:

```python
# Convolve the window to get back one output neuron
Z[i, h, w, c] = ...
# Apply activation
A[i, h, w, c] = activation(Z[i, h, w, c])
```

You don't need to do it here.

### 3.1 - Convolutional neural networks - backward pass (OPTIONAL / UNGRADED)

When implementing a simple neural network, you used backpropagation to compute the derivatives with respect to the cost to update the parameters. Similarly, when implementing convolutional neural networks you need to calculate the derivatives with respect to the loss prior to updating the parameters. The following equations are not trivial and have not been derived in lecture:

#### 3.1.1 - Computing dA:
This is the formula for computing $dA$ with respect to the loss for a certain filter c and a given training example:

$$ dA += \sum _{h=0} ^{n_H} \sum_{w=0} ^{n_W} W_c \times dZ_{hw} \tag{1}$$

Where $W_c$ is a filter and $dZ_{hw}$ is a scalar corresponding to the gradient of the cost with respect to the output of the conv layer Z at the hth row and wth column (corresponding to the dot product taken at the ith stride left and jth stride down). Note that at each time, we multiply the the same filter $W_c$ by a different dZ when updating dA. We do so mainly because when computing the forward propagation, each filter is dotted and summed by a different a_slice. Therefore when computing the backprop for dA, we are just adding the gradients of all the a_slices. 

When coding, inside the proper loops, this formula translates into:
```python
da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
```

#### 3.1.2 - Computing dW:
This is the formula for computing $dW_c$ ($dW_c$ is the derivative of one filter) with respect to the loss:

$$ dW_c  += \sum _{h=0} ^{n_H} \sum_{w=0} ^ {n_W} a_{slice} \times dZ_{hw}  \tag{2}$$

Where $a_{slice}$ corresponds to the slice which was used to generate the acitivation $Z_{ij}$. Hence, this ends up giving us the gradient for $W$ with respect to that slice. Since it is the same $W$, we will just add up all such gradients to get $dW$. 

When coding, inside the proper loops, this formula translates into:
```python
dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
```

#### 3.1.3 - Computing db:

This is the formula for computing $db$ with respect to the loss for a certain filter f:

$$ db = \sum_h \sum_w dZ_{hw} \tag{3}$$

As you have previously seen in basic neural networks, db is taken by summing $dZ$. In this case, you are just summing over all the gradients of the conv output (Z) with respect to the loss. 

When coding, inside the proper loops, this formula translates into:
```python
db[:,:,:,c] += dZ[i, h, w, c]
```

**Exercise**: Implement the `conv_backward` function below. You should sum over all the training examples, filters, heights, and widths. You should then compute the derivatives using formulas 1, 2 and 3 above. 

In [ ]:
# GRADED FUNCTION: conv_backward

def conv_backward(dZ, cache):
    """
    Implements the backward propagation for a convolution function
    
    Arguments:
    dZ -- gradient of the cost with respect to the output of the conv layer (Z), numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward(), output of conv_forward()
    
    Returns:
    dA_prev -- gradient of the cost with respect to the input of the conv layer (A_prev),
               numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    dW -- gradient of the cost with respect to the weights of the conv layer (W)
          numpy array of shape (f, f, n_C_prev, n_C)
    db -- gradient of the cost with respect to the biases of the conv layer (b)
          numpy array of shape (1, 1, 1, n_C)
    """
    
    ### START CODE HERE ###
    # Retrieve information from "cache"
    (A_prev, W, b, hparameters) = None
    
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = None
    
    # Retrieve information from "hparameters"
    stride = None
    pad = None
    
    # Retrieve dimensions from dZ's shape
    (m, n_H, n_W, n_C) = None
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = None                          
    dW = None
    db = None 

    # Pad A_prev and dA_prev
    A_prev_pad = None
    dA_prev_pad = None
    
    for i in range(m):                          # loop over the training examples
        
        # select ith training example from A_prev_pad and dA_prev_pad
        a_prev_pad = None
        da_prev_pad = None
        
        for h in range(n_H):                    # loop over vertical axis of the output volume
            for w in range(n_W):                # loop over horizontal axis of the output volume
                for c in range(n_C):            # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Use the corners to define the slice
                    a_slice = None

                    # Update gradients for the window and the filter's parameters using the code given above
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += None
                    dW[:,:,:,c] += None
                    db[:,:,:,c] += None
                    
        # set the ith training example's dA_prev to the unpaded da_prev_pad (Hint: use X[pad:-pad, pad:-pad, :])
        dA_prev[i, :, :, :] = None
        
    ### END CODE HERE ###
    
    return dA_prev, dW, db

In [ ]:
np.random.seed(1)
dA, dW, db = conv_backward(Z, cache)
print("dA_mean =", np.mean(dA))
print("dW_mean =", np.mean(dW))
print("db_mean =", np.mean(db))

** Expected Output: **
<table>
    <tr>
        <td>
            **dA_mean**
        </td>
        <td>
            -21.7047022204
        </td>
    </tr>
    <tr>
        <td>
            **dW_mean**
        </td>
        <td>
            32.554484381296923
        </td>
    </tr>
    <tr>
        <td>
            **db_mean**
        </td>
        <td>
            1272.10348876
        </td>
    </tr>

</table>


## 4 - Convolutional neural networks - Pooling

The pooling (POOL) layer is a layer we often use right after the CONV layer. It is used to reduce the height and width of the input to shrink the spatial dimensions. Two advantages of the POOL layer are:

- Reduces the amount of computation.
- Reduces the number of parameters in the next layer, which makes it easier to train and less likely to overfit.

Conversely, one disadvantage of the POOL layer is the loss of spatial information. Therefore, pooling is rarely used in models that detect objects. The two types of pooling layers are: 

- Max-pooling layer: slides an ($f, f$) window over the input and stores the max value of the window in another matrix.

- Average-pooling layer: slides an ($f, f$) window over the input and stores the average value of the window in another matrix.

<table>
<td>
<img src="images/max_pool1.png" style="width:500px;height:300px;">
<td>

<td>
<img src="images/a_pool.png" style="width:500px;height:300px;">
<td>
</table>

Each different color represents a window. Depending on whether we are doing MAX-POOL or AVG-POOL we would output the corresponding number in a different matrix. As a result, we have fewer parameters to deal with. 

### 4.1 - Forward Pooling
Now, you are going to implement MAX->POOL and AVG->POOL, in the same function. 

**Exercise**: Implement the forward pass of the pooling layer. 

In [ ]:
# GRADED FUNCTION: pool_forward

def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparams 
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Retrieve hyperparameters from "hparameters"
    f = None
    stride = None
    
    # Define the dimensions of the output
    n_H = None
    n_W = None
    n_C = None
    
    # Initialize output matrix A
    A = None             
    
    for i in range(m):                            # loop over the training examples
        for h in range(n_H):                      # loop on the vertical axis of the output volume
            for w in range(n_W):                  # loop on the horizontal axis of the output volume
                for c in range (n_C):             # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Use the corners to define the current slice
                    a_prev_slice = None
                    
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                    if None:
                        A[i, h, w, c] = None
                    elif None:
                        A[i, h, w, c] = None
    
    ### END CODE HERE ###
    
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    
    return A, cache

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(2, 4, 4, 3)
hparams = {"stride" : 1,
           "f": 4}
A, cache = pool_forward(A_prev, hparams)
print("A =", A)

**Expected Output:**
<table>

    <tr>
    <td>
    Z  =
    </td>
        <td>
         [[[[ 1.74481176  1.6924546   2.10025514]]] <br/>


 [[[ 1.19891788  1.51981682  2.18557541]]]]

        </td>
    </tr>

</table>


### 4.2 Backward Pooling (OPTIONAL)

Before jumping into the backpropagation of the pooling layer, you are going to build a helper function called `create_mask_from_window()` which does the following: 

$$ X = \begin{bmatrix}
1 && 3 \\
4 && 2
\end{bmatrix} \quad \rightarrow  \quad M =\begin{bmatrix}
0 && 0 \\
1 && 0
\end{bmatrix}\tag{4}$$

As you can see, this function creates a "mask" matrix which keeps track of where the maximum of the matrix is. True (1) indicates the position of the maximum in X, the other entries are False (0).

**Exercise**: Implement `create_mask_from_window()`. This function will be helpful for pooling backward. Hints:
- [np.max()]() may be helpful. It computes the maximum of an array.
- If you have a matrice X and a scalar x: `A = (X == x)` will return a matrix A of the same size as X such that:
```
A[i,j] = True if X[i,j] = x
A[i,j] = False if X[i,j] != x
```
- Here, you don't need to consider cases where there are several maxima in a matrix.

In [ ]:
# GRADED FUNCTION: create_mask_from_window

def create_mask_from_window(x):
    """
    Creates a mask from an input matrix x, to identify the max entry of x.
    
    Arguments:
    x -- Array of shape (f, f, n_C)
    
    Returns:
    mask -- Array of the same shape as window, contains a True at the position corresponding to the max entry of x.
    """
    
    ### START CODE HERE ### (1 line)
    mask = None
    ### END CODE HERE ###
    
    return mask

In [ ]:
np.random.seed(1)
x = np.random.randn(2,2,2)
mask = create_mask_from_window(x)
print('x = ',x)
print("mask = ", mask)

**Expected Output:** 

<table> 
<tr> 
<td>

**x =**
</td>

<td>

[[[ 1.62434536 -0.61175641]
  [-0.52817175 -1.07296862]]<br\> 

 [[ 0.86540763 -2.3015387 ]
  [ 1.74481176 -0.7612069 ]]]

  </td>
</tr>

<tr> 
<td>
**mask =**
</td>
<td>
[[[False False]
  [False False]] <br\> 

 [[False False]
  [ True False]]]
</td>
</tr>


</table>

Intuitively, remember that backpropagation is about influence. Anything that has an influence on the output of the forward propagation should appear in the backprop, so we need to know where the "max" of the MAXPOOL came from.

### 4.2.1 - Value distribution (OPTIONAL)

When backpropagating, you will have to distribute the value of an activation to a window so that the window could be of the same dimension as the filter. Remember that when you were pooling forward (mode = 'average'), you only kept the average number of the matrix. Similarly, when pooling backward, you will have to get a number and distribute it through a matrix of the same dimension as the filter. For example if we did average pooling in the forward pass then the backward pass will look like: 
$$ dZ = 4 \quad \rightarrow  \quad dZ =\begin{bmatrix}
1/4 && 1/4 \\
1/4 && 1/4
\end{bmatrix}\tag{5}$$

This implies that each spot in the $dZ$ matrix contributes equally to output because in the forward pass, we took an average, hence each element had the same weight.

**Exercise**: Implement the function below to equally distribute a value dz through a matrix of dimension shape. [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ones.html)

In [ ]:
# GRADED FUNCTION: distribute_value

def distribute_value(dz, shape):
    """
    Distributes the input value in the matrix of dimension shape
    
    Arguments:
    dz -- input scalar
    shape -- the shape (n_H, n_W) of the output matrix for which we want to distribute the value of dz
    
    Returns:
    a -- Array of size (n_H, n_W) for which we distributed the value of dz
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from shape
    (n_H, n_W) = None
    
    # Compute the value to distribute on the matrix
    average = None
    
    # Create a matrix where every entry is the "average" value
    a = None
    ### END CODE HERE ###
    
    return a

In [ ]:
a = distribute_value(2, (2,2))
print('distributed value =', a)

**Expected Output**: 

<table> 
<tr> 
<td>
distributed_value =
</td>
<td>
[[ 0.5  0.5]
<br\> 
[ 0.5  0.5]]
</td>
</tr>
</table>

### 4.2.2 - Pooling backward (OPTIONAL)

You now have everything you need to compute the backward propagation of a pooling layer.

**Exercise**: Implement the `pool_backward` function in both modes (`"max"` and `"average"`). You will once again use 4 forloops (training examples, height, width, and channels). You should use an `if/elif` statement to see if the mode is equal to `'max'` or `'average'`. If it is equal to 'average' you should use the `distribute_value()` function you implemented above to create a matrix of the same shape as `a_slice`. Else, the mode is equal to `max`, you will create a mask with `create_mask_from_window()` and multiply it by the corresponding value of dZ.

In [ ]:
# GRADED FUNCTION: pool_backward

def pool_backward(dA, cache, mode = "max"):
    """
    Implements the backward pass of the pooling layer
    
    Arguments:
    dA -- gradient of cost with respect to the output of the pooling layer, same shape as A
    cache -- cache output from the forward pass of the pooling layer, contains the layer's input and hparams 
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    dA_prev -- gradient of cost with respect to the input of the pooling layer, same shape as A_prev
    """
    
    ### START CODE HERE ###
    
    # Retrieve information from cache
    (A_prev, hparams) = None
    
    # Retrieve hyperparameters from "hparams"
    stride = None
    f = None
    
    # Retrieve dimensions from A_prev's shape and dA's shape
    m, n_H_prev, n_W_prev, n_C_prev = None
    m, n_H, n_W, n_C = None
    
    # Initialize dA_prev with zeros
    dA_prev = None
    
    for i in range(m): # loop over the training examples
        
        # select training example from A_prev
        a_prev = None
        
        for h in range(n_H):                    # loop on the vertical axis
            for w in range(n_W):                # loop on the horizontal axis
                for c in range(n_C):            # loop over the channels (depth)
                    
                    # Find the corners of the current "slice"
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Compute the backward propagation in both modes.
                    if None:
                        # Use the corners and "c" to define the current slice 
                        a_prev_slice = None
                        # Select the max of the a_prev_slice to create a mask
                        mask = None
                        # Set dA_prev to be the mask multiplied by the correct value of dA
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] = None
                        
                    elif None:
                        
                        # Get the value a from dA
                        da = None
                        # Define the shape of the filter
                        shape = None
                        # Distribute it to get the correct slice of dA_prev
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] = None
                        
    ### END CODE ###
    
    return dA_prev

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(5, 5, 3, 2)
hparameters = {"stride" : 1, "f": 2}
A, cache = pool_forward(A_prev, hparameters)
dA = np.random.randn(5, 4, 2, 2)

dA_prev = pool_backward(dA, cache, mode = "max")
print("mode = max")
print('mean of dA = ', np.mean(dA))
print('mean of dA_prev = ', np.mean(dA_prev))  
print()

dA_prev = pool_backward(dA, cache, mode = "average")
print("mode = average")
print('mean of dA = ', np.mean(dA))
print('mean of dA_prev = ', np.mean(dA_prev))  

**Expected Output**: 

mode = max:
<table> 
<tr> 
<td>

**mean of dA =**
</td>

<td>

0.145713902729

  </td>
</tr>

<tr> 
<td>
**mean of dA_prev =** 
</td>
<td>
0.0113681444994
</td>
</tr>
</table>

mode = average
<table> 
<tr> 
<td>

**mean of dA =**
</td>

<td>

0.145713902729

  </td>
</tr>

<tr> 
<td>
**mean of dA_prev =** 
</td>
<td>
0.0298114155589
</td>
</tr>
</table>

### Congratulations !

Congratulation on completing this assignment. You now understand how convolutional neural networks work. You have implemented all the building blocks of a neural network. In the next assignment you will implement a ConvNet using tensorflow.